<a href="https://colab.research.google.com/github/benjaminbrown038/torch/blob/main/torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

from torch.nn import functional
from torch.nn import Sequential, Conv2d, MaxPool2d, Dropout, Flatten, Linear, Softmax, CrossEntropyLoss
from torch.optim import SGD, Adam
from torch import optim
from torch.utils.data import DataLoader
from torch.optim import SGD


import torchvision

from torchvision import models
from torchvision import transforms
from torchvision.models import resnet50, alexnet, alexnet, inception_v3
from torch.nn import Sequential

from torchsummary import summary

import matplotlib.pyplot as plt
import numpy
import os
import cv2

In [11]:
%%capture

mnist_train = torchvision.datasets.MNIST(root = '/data', download = True,train = True,transform = transforms.ToTensor())
mnist_test = torchvision.datasets.MNIST(root = '/data', download = True, train = False, transform = transforms.ToTensor())

mnist_train_dataloader = DataLoader(mnist_train,batch_size = 64,shuffle = True)
mnist_test_dataloader = DataLoader(mnist_test,batch_size = 64, shuffle = True)


cifar_train = torchvision.datasets.MNIST(root = '/data', download = True, train = True)
cifar_test = torchvision.datasets.MNIST(root = '/data',download = True, train = False)


cifar_train_dataloader = DataLoader(mnist_train, batch_size = 64, shuffle = True)
cifar_test_dataloader = DataLoader(mnist_test, batch_size = 64, shuffle = True)


In [2]:
model = Sequential(
            Conv2d(1, 32, kernel_size=3),
            Conv2d(32, 64, kernel_size=3),
            MaxPool2d(2),
            Dropout(0.25),
            Flatten(),
            Linear(9216, 128),
            Linear(128, 10),
            Softmax())
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
         MaxPool2d-3           [-1, 64, 12, 12]               0
           Dropout-4           [-1, 64, 12, 12]               0
           Flatten-5                 [-1, 9216]               0
            Linear-6                  [-1, 128]       1,179,776
            Linear-7                   [-1, 10]           1,290
           Softmax-8                   [-1, 10]               0
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.66
Params size (MB): 4.58
Estimated Total Size (MB): 5.24
----------------------------------------------------------------


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [3]:
model1 = Sequential(
		Conv2d(1,32,kernel_size = 2),
		MaxPool2d(2,2),
		Conv2d(32,64,kernel_size = 2),
		MaxPool2d(2,2),
		Conv2d(64,128,kernel_size = 2),
		Dropout(0.25),
		MaxPool2d(1,1),
		Conv2d(128,10,kernel_size = 3),
		Flatten(),
		Linear(90,10))
summary(model1,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 27, 27]             160
         MaxPool2d-2           [-1, 32, 13, 13]               0
            Conv2d-3           [-1, 64, 12, 12]           8,256
         MaxPool2d-4             [-1, 64, 6, 6]               0
            Conv2d-5            [-1, 128, 5, 5]          32,896
           Dropout-6            [-1, 128, 5, 5]               0
         MaxPool2d-7            [-1, 128, 5, 5]               0
            Conv2d-8             [-1, 10, 3, 3]          11,530
           Flatten-9                   [-1, 90]               0
           Linear-10                   [-1, 10]             910
Total params: 53,752
Trainable params: 53,752
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.38
Params size (MB): 0.21
Estimated Tot

In [4]:
model2 = Sequential(
		Conv2d(1,32,kernel_size = 3),
		Conv2d(32,64,kernel_size = 3),
		MaxPool2d(2),
		Dropout(0.25),
	  Flatten(),
		Linear(9216,128),
		Linear(128,10))
summary(model2,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
         MaxPool2d-3           [-1, 64, 12, 12]               0
           Dropout-4           [-1, 64, 12, 12]               0
           Flatten-5                 [-1, 9216]               0
            Linear-6                  [-1, 128]       1,179,776
            Linear-7                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.66
Params size (MB): 4.58
Estimated Total Size (MB): 5.24
----------------------------------------------------------------


In [5]:
%%capture
pre_trained_one = resnet50(pretrained = True)

In [6]:
%%capture
pre_trained_two = inception_v3(pretrained=True)

In [7]:
%%capture
pre_trained_three = alexnet(pretrained = True)

In [15]:
def train_loop(dataloader,model,loss_fn,optimizer):
	size = len(dataloader.dataset)
	for batch, (x,y) in enumerate(dataloader):
		pred = model(x)
		loss = loss_fn(pred,y)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		if batch % 100 == 0:
			loss,current = loss.item(), batch*len(x)
		print(loss,current)

In [17]:
loss = CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr = 0.001,momentum = 0.9)
optimizer1 = optim.SGD(model1.parameters(),lr=0.001,momentum = 0.9)
optimizer2 = optim.SGD(model2.parameters(), lr =0.001,momentum = 0.9)
optimizer3 = optim.SGD(pre_trained_one.parameters(),lr=0.001,momentum = 0.9)
optimizer4 = optim.SGD(pre_trained_two.parameters(),lr=0.001,momentum=0.9)
optimizer5 = optim.SGD(pre_trained_three.parameters(),lr=0.001,momentum=0.9)

In [16]:
train_loop(mnist_train_dataloader,model,loss,optimizer)

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


1.5886682271957397 0
tensor(1.5683, grad_fn=<NllLossBackward0>) 0
tensor(1.5595, grad_fn=<NllLossBackward0>) 0
tensor(1.5341, grad_fn=<NllLossBackward0>) 0
tensor(1.6065, grad_fn=<NllLossBackward0>) 0
tensor(1.5914, grad_fn=<NllLossBackward0>) 0
tensor(1.5601, grad_fn=<NllLossBackward0>) 0
tensor(1.5719, grad_fn=<NllLossBackward0>) 0
tensor(1.5382, grad_fn=<NllLossBackward0>) 0
tensor(1.5610, grad_fn=<NllLossBackward0>) 0
tensor(1.5721, grad_fn=<NllLossBackward0>) 0
tensor(1.5200, grad_fn=<NllLossBackward0>) 0
tensor(1.5629, grad_fn=<NllLossBackward0>) 0
tensor(1.5323, grad_fn=<NllLossBackward0>) 0
tensor(1.5533, grad_fn=<NllLossBackward0>) 0
tensor(1.6023, grad_fn=<NllLossBackward0>) 0
tensor(1.5066, grad_fn=<NllLossBackward0>) 0
tensor(1.5599, grad_fn=<NllLossBackward0>) 0
tensor(1.5523, grad_fn=<NllLossBackward0>) 0
tensor(1.6275, grad_fn=<NllLossBackward0>) 0
tensor(1.6398, grad_fn=<NllLossBackward0>) 0
tensor(1.4966, grad_fn=<NllLossBackward0>) 0
tensor(1.6378, grad_fn=<NllLossBac

In [19]:
train_loop(mnist_train_dataloader,model1,loss,optimizer)

2.3057734966278076 0
tensor(2.3069, grad_fn=<NllLossBackward0>) 0
tensor(2.2969, grad_fn=<NllLossBackward0>) 0
tensor(2.3188, grad_fn=<NllLossBackward0>) 0
tensor(2.3144, grad_fn=<NllLossBackward0>) 0
tensor(2.3126, grad_fn=<NllLossBackward0>) 0
tensor(2.2879, grad_fn=<NllLossBackward0>) 0
tensor(2.2838, grad_fn=<NllLossBackward0>) 0
tensor(2.3106, grad_fn=<NllLossBackward0>) 0
tensor(2.2921, grad_fn=<NllLossBackward0>) 0
tensor(2.3074, grad_fn=<NllLossBackward0>) 0
tensor(2.2984, grad_fn=<NllLossBackward0>) 0
tensor(2.3068, grad_fn=<NllLossBackward0>) 0
tensor(2.3046, grad_fn=<NllLossBackward0>) 0
tensor(2.2986, grad_fn=<NllLossBackward0>) 0
tensor(2.2967, grad_fn=<NllLossBackward0>) 0
tensor(2.3077, grad_fn=<NllLossBackward0>) 0
tensor(2.3131, grad_fn=<NllLossBackward0>) 0
tensor(2.2913, grad_fn=<NllLossBackward0>) 0
tensor(2.3136, grad_fn=<NllLossBackward0>) 0
tensor(2.3139, grad_fn=<NllLossBackward0>) 0
tensor(2.2941, grad_fn=<NllLossBackward0>) 0
tensor(2.3150, grad_fn=<NllLossBac

In [22]:
train_loop(mnist_train_dataloader,model2,loss,optimizer)

2.301766872406006 0
tensor(2.3234, grad_fn=<NllLossBackward0>) 0
tensor(2.3043, grad_fn=<NllLossBackward0>) 0
tensor(2.2948, grad_fn=<NllLossBackward0>) 0
tensor(2.3160, grad_fn=<NllLossBackward0>) 0
tensor(2.2925, grad_fn=<NllLossBackward0>) 0
tensor(2.3097, grad_fn=<NllLossBackward0>) 0
tensor(2.3148, grad_fn=<NllLossBackward0>) 0
tensor(2.3158, grad_fn=<NllLossBackward0>) 0
tensor(2.3227, grad_fn=<NllLossBackward0>) 0
tensor(2.3039, grad_fn=<NllLossBackward0>) 0
tensor(2.3265, grad_fn=<NllLossBackward0>) 0
tensor(2.2951, grad_fn=<NllLossBackward0>) 0
tensor(2.3259, grad_fn=<NllLossBackward0>) 0
tensor(2.3104, grad_fn=<NllLossBackward0>) 0
tensor(2.2981, grad_fn=<NllLossBackward0>) 0
tensor(2.3120, grad_fn=<NllLossBackward0>) 0
tensor(2.2996, grad_fn=<NllLossBackward0>) 0
tensor(2.3106, grad_fn=<NllLossBackward0>) 0
tensor(2.2997, grad_fn=<NllLossBackward0>) 0
tensor(2.3059, grad_fn=<NllLossBackward0>) 0
tensor(2.3258, grad_fn=<NllLossBackward0>) 0
tensor(2.2850, grad_fn=<NllLossBack

In [25]:
train_loop(mnist_train_dataloader,pre_trained_one,loss,optimizer)

RuntimeError: ignored

In [28]:
train_loop(mnist_train_dataloader,pre_trained_two,loss,optimizer)

IndexError: ignored

In [29]:
train_loop(mnist_train_dataloader,pre_trained_three,loss,optimizer)

RuntimeError: ignored